In [5]:
!unzip e_doctor_lora.zip -d e_doctor_lora/


Archive:  e_doctor_lora.zip
   creating: e_doctor_lora/e_doctor_lora/
  inflating: e_doctor_lora/e_doctor_lora/special_tokens_map.json  
  inflating: e_doctor_lora/e_doctor_lora/adapter_model.safetensors  
  inflating: e_doctor_lora/e_doctor_lora/chat_template.jinja  
  inflating: e_doctor_lora/e_doctor_lora/tokenizer_config.json  
  inflating: e_doctor_lora/e_doctor_lora/adapter_config.json  
  inflating: e_doctor_lora/e_doctor_lora/tokenizer.json  
  inflating: e_doctor_lora/e_doctor_lora/README.md  


In [9]:
!pip install torch transformers bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.2 MB/s eta 0:00:00


In [11]:

!pip install unsloth # install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Also get the latest version Unsloth!

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.6/275.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-ydmwxt3e
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-ydmwxt3e
  Resolved https://github.com/unslothai/unsloth.git to commit 3340eaa41ce58619daf79d2783e49a45f8553a61
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.5.10-py3-none-any.whl size=276844 sha256=8d26dd8e0ea3f0ba223462690cead7ccff42bc7546e7e7a1ef6749a933dca4bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-uutmt2vp/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.5.9
    Uninstalling unsloth-2025.5.9:
      Successfully uninstalled unsloth-2025.5.9


In [1]:
# Step3: Import necessary libraries
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"  # Your base model
max_seq_length = 2048
load_in_4bit = True

# Load base model + tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    dtype = None,  # or torch.float16 / torch.bfloat16
)

# Apply LoRA weights from your saved adapter
model.load_adapter("/content/e_doctor_lora/e_doctor_lora")  # ✅ Loads your saved fine-tuned adapter


==((====))==  Unsloth 2025.5.10: Fast Llama patching. Transformers: 4.52.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [9]:
import torch
import math

def calculate_perplexity(model, tokenizer, texts, device="cuda"):
    model.eval()
    # model.to(device)
    perplexities = []
    with torch.no_grad():
        for text in texts:
            encoding = tokenizer(text, return_tensors="pt")
            input_ids = encoding.input_ids.to(device)
            # Some Unsloth models may require attention_mask
            attention_mask = encoding.get("attention_mask", None)
            if attention_mask is not None:
                attention_mask = attention_mask.to(device)
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            else:
                outputs = model(input_ids=input_ids, labels=input_ids)
            # Handle output as tuple or object
            loss = outputs.loss if hasattr(outputs, "loss") else outputs[0]
            ppl = math.exp(loss.item())
            perplexities.append(ppl)
    return perplexities

# Example usage:
test_texts = [
    "the patient has symptoms of acute myocardial infarction and needs urgent care.",

]

device = "cuda" if torch.cuda.is_available() else "cpu"
ppls = calculate_perplexity(model, tokenizer, test_texts, device)
print(ppls)


[36.11551720221854]


In [18]:
import json
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

# 1. Load your test data
with open('/content/test3.json', 'r') as f:
    test_data = json.load(f)

# # 2. Load base model + tokenizer
# model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
# max_seq_length = 2048
# load_in_4bit = True

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = model_name,
#     max_seq_length = max_seq_length,
#     load_in_4bit = load_in_4bit,
#     dtype = None,  # or torch.float16 / torch.bfloat16
# )

# # 3. Load your LoRA adapter weights
# model.load_adapter("/content/e_doctor_lora/e_doctor_lora")  # Path to your LoRA adapter

# 4. Device setup
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Do NOT call model.to(device) for quantized models!

# 5. Generation function
def generate_answer(question, tokenizer, model, device='cuda'):
    prompt = f"Patient: {question}\nDoctor:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=128,
            do_sample=False
        )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the doctor's answer
    if "Doctor:" in decoded:
        answer = decoded.split("Doctor:")[-1].strip()
    else:
        answer = decoded.strip()
    return answer

# 6. Generate answers for your test set
references = []
candidates = []

for item in test_data:
    question = item['input']
    reference = item['output']
    candidate = generate_answer(question, tokenizer, model, device)
    references.append(reference)
    candidates.append(candidate)

# Now references and candidates contain your ground-truth and model-generated answers.
# You can use them for ROUGE or other evaluation.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

In [19]:
for ref, cand in zip(references[:5], candidates[:5]):
    print("Reference:", ref)
    print("Candidate:", cand)
    print("-" * 40)


Reference: symptoms are non specific nothing to worry it is probably anxiety or stress.the recent neck burning sensation is irrelevant and u need to notice if any other symptoms or signs appears
Candidate: Hi, welcome to HCM. I have reviewed your query. You are 20 years old. You have been having nausea and burning sensation in the neck for the last 2 days. You should consult a doctor for a proper diagnosis and treatment. Hope I have answered your query. Let me know if I can assist you further. Take care.

*Disclaimer: This response is generated for informational purposes only and is not a substitute for professional medical advice, diagnosis, or treatment. Please consult a licensed healthcare provider for personalized care.*
----------------------------------------
Reference: Why Sacroilitis and Hip Arthritis together? I think you need to consult an orthopedic doctor , first get an x ray Pelvis with both hips and some blood tests. You have not mentioned your age also
Candidate: Hi, wel

In [20]:
!pip install rouge-score

from rouge_score import rouge_scorer
import numpy as np

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge1_scores, rouge2_scores, rougeL_scores = [], [], []

for ref, cand in zip(references, candidates):
    scores = scorer.score(ref, cand)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

print(f"Average ROUGE-1: {np.mean(rouge1_scores):.4f}")
print(f"Average ROUGE-2: {np.mean(rouge2_scores):.4f}")
print(f"Average ROUGE-L: {np.mean(rougeL_scores):.4f}")


Average ROUGE-1: 0.1811
Average ROUGE-2: 0.0212
Average ROUGE-L: 0.1050


In [21]:
import torch
import time

def measure_latency(model, tokenizer, prompt, device='cuda', num_runs=10):

    inputs = tokenizer(prompt, return_tensors='pt').to(device)

    for _ in range(3):
        _ = model.generate(**inputs, max_new_tokens=50)

    torch.cuda.synchronize() if device == "cuda" else None
    start_time = time.time()
    for _ in range(num_runs):
        _ = model.generate(**inputs, max_new_tokens=50)
    torch.cuda.synchronize() if device == "cuda" else None
    end_time = time.time()
    avg_latency = (end_time - start_time) / num_runs
    return avg_latency * 1000


prompt = "Patient: I have a headache and fever. What should I do?\nDoctor:"
device = "cuda" if torch.cuda.is_available() else "cpu"
avg_latency = measure_latency(model, tokenizer, prompt, device)
print(f'Average inference latency (ms): {avg_latency:.2f}')


Average inference latency (ms): 5157.79


In [22]:
import os

def get_model_size(model_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(model_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)

model_path = '/content/e_doctor_lora/e_doctor_lora'
model_size_mb = get_model_size(model_path)
print(f"Model size (MB): {model_size_mb:.2f}")


Model size (MB): 176.53


In [24]:
import pandas as pd

metrics = ['ROUGE-1', 'ROUGE-2', 'PPL', 'Latency (ms)', 'Model Size (MB)']
values = [0.1811, 0.0212, 36.11551720221854, 5157.7, 176.53]

description = [
    'Unigram overlap (measures recall of important words)',
    'Bigram overlap (measures recall of important word pairs)',
    'reflects model fluency and prediction quality',
    'Inference time per token',
    'Disk size of the fine-tuned model (smaller is more deployable)'
]

comparative_results_df = pd.DataFrame({
    'Metric': metrics,
    'Value': values,
    'Description': description
})


print(comparative_results_df.to_markdown(index=False))


| Metric          |     Value | Description                                                    |
|:----------------|----------:|:---------------------------------------------------------------|
| ROUGE-1         |    0.1811 | Unigram overlap (measures recall of important words)           |
| ROUGE-2         |    0.0212 | Bigram overlap (measures recall of important word pairs)       |
| PPL             |   36.1155 | reflects model fluency and prediction quality                  |
| Latency (ms)    | 5157.7    | Inference time per token                                       |
| Model Size (MB) |  176.53   | Disk size of the fine-tuned model (smaller is more deployable) |
